In [1]:
#### python의 slicing ####

import numpy as np

a = np.array([1, 2, 3, 4, 5])

print(a[1:3]) # 1번째부터 3번째(제외)까지
print(a[-1]) # -1은 마지막 원소
a[0:2] = 9 # 0번째부터 1번째까지
print(a)

print()

b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

print(b[:, 1]) # 모든 행의 1번째 원소
print(b[-1]) # 마지막 행
print(b[-1, :]) # 마지막 행
print(b[-1, ...]) # :을 ...으로 표현 가능
print(b[0:2, :]) # 0~1행
print(b[:, 0:-1]) # 모든 행의 마지막 원소를 제외한 행렬

[2 3]
5
[9 9 3 4 5]

[ 2  6 10]
[ 9 10 11 12]
[ 9 10 11 12]
[ 9 10 11 12]
[[1 2 3 4]
 [5 6 7 8]]
[[ 1  2  3]
 [ 5  6  7]
 [ 9 10 11]]


In [2]:
#### 파일에서 데이터 읽어오기 ####
import tensorflow as tf
import numpy as np
tf.set_random_seed(777) # for reproducibility

# 파일에서 데이터 읽어오기
xy = np.loadtxt('data/data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1] # 마지막 열을 제외한 데이터
y_data = xy[:, [-1]] # 마지막 열 (-1만 쓰면 한 행으로 반환)

# 데이터 확인
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], 
                                   feed_dict={X: x_data, Y: y_data})
    if step % 100 == 0:
        print(step, "\nCost:", cost_val, "\nPrediction:\n", hy_val, "\n")

# 테스트 : 내 점수 예상
print("Your score will be", sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))
print("Other scores will be", sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

(6, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]] 6
(6, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]]
0 
Cost: 20344.273 
Prediction:
 [[22.048063 ]
 [21.619787 ]
 [24.096693 ]
 [22.293005 ]
 [18.633902 ]
 [ 7.2669735]] 

100 
Cost: 6.3537693 
Prediction:
 [[155.79639]
 [182.60619]
 [182.59769]
 [194.92845]
 [141.45721]
 [ 97.87128]] 

200 
Cost: 6.034921 
Prediction:
 [[155.69016 ]
 [182.65692 ]
 [182.55382 ]
 [194.90393 ]
 [141.5211  ]
 [ 97.975174]] 

300 
Cost: 5.736275 
Prediction:
 [[155.5874 ]
 [182.70596]
 [182.51134]
 [194.88031]
 [141.58273]
 [ 98.0758 ]] 

400 
Cost: 5.456492 
Prediction:
 [[155.48799 ]
 [182.75337 ]
 [182.47021 ]
 [194.8576  ]
 [141.6422  ]
 [ 98.173256]] 

500 
Cost: 5.194362 
Prediction:
 [[155.39178]
 [182.79918]
 [182.4304 ]
 [194.83575]
 [141.69955]
 [ 98.26765]] 

600 
Cost: 4.9487553 
Prediction:
 [[155.29875]
 [182.84352]
 [182.39188]
 [194.81474]
 [141.75491]
 [ 98.35909]] 

70

In [2]:
#### Queue runners 이용 ####
# 파일이 너무 커서 메모리에 한번에 담을 수 없을 때 사용
import tensorflow as tf

# 파일 읽어오기
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue') # 배열에 여러개 파일 가능
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# 결과 디코딩
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# csv의 batch 수집
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], 
                                   feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "\nCost:", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

# 실행 안됨. 원인은 아직 모르겠음..

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Expect 4 fields but have 3 in record 0
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], field_delim=",", na_value="", use_quote_delim=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReaderReadV2:1, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0)]]


OutOfRangeError: FIFOQueue '_1_batch/fifo_queue' is closed and has insufficient elements (requested 10, current size 0)
	 [[Node: batch = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](batch/fifo_queue, batch/n)]]

Caused by op 'batch', defined at:
  File "C:\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Anaconda3\envs\tensorflow\lib\site-packages\tornado\platfor